In [11]:
import urllib, urllib.request    
import requests
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
from requests.adapters import TimeoutSauce
from datetime import datetime
import os
import feedparser
try:
    from urllib.request import urlopen
except ImportError:
    from urllib2 import urlopen

In [3]:
start_date = datetime(2023, 3, 1)
end_date = datetime(2023, 3, 8)

start = start_date.strftime("%Y%m%d")
end = end_date.strftime("%Y%m%d")
range_str = f"[{start}000000+TO+{end}000000]"
range_query = f"lastUpdatedDate:{range_str}"

range_query

'lastUpdatedDate:[20230301000000+TO+20230308000000]'

In [4]:
subcat = ["GA", "CO", "EP", "HE", "IM", "SR"]

cat_query = "%28"  # open parenthesis
for n, s in enumerate(subcat):
    cat_query += f"astro-ph.{s}"
    if n < len(subcat)-1:
        cat_query += "+OR+"
    else:
        cat_query += "%29"  # close parenthesis
        
cat_query

'%28astro-ph.GA+OR+astro-ph.CO+OR+astro-ph.EP+OR+astro-ph.HE+OR+astro-ph.IM+OR+astro-ph.SR%29'

In [5]:
base_url = "http://export.arxiv.org/api/query?"
sort_query = f"max_results={999}&sortBy=submittedDate&sortOrder=descending"

full_query = f"search_query={cat_query}+AND+{range_query}&{sort_query}"

url = base_url + full_query
url

'http://export.arxiv.org/api/query?search_query=%28astro-ph.GA+OR+astro-ph.CO+OR+astro-ph.EP+OR+astro-ph.HE+OR+astro-ph.IM+OR+astro-ph.SR%29+AND+lastUpdatedDate:[20230301000000+TO+20230308000000]&max_results=999&sortBy=submittedDate&sortOrder=descending'

In [17]:
response = urlopen(url).read()
feed = feedparser.parse(response)

In [18]:
pdf_links = []
for e in feed.entries:
    for l in e.links:
        if "pdf" in l.href:
            pdf_links.append(l.href)
            
print(len(pdf_links)) # number changes for given url!

515


In [19]:
directory = ".//data//arxiv test//"
files = os.listdir(directory)

headers = {
    'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.111 Safari/537.36',
}
session = requests.Session()
retry = Retry(connect=3, backoff_factor=10)
adapter = HTTPAdapter(max_retries=retry)
session.mount('http://', adapter)
session.mount('https://', adapter)

for link in pdf_links:
    if f"{link[21:]}.pdf" in files:
        continue
        
    response = session.get(link, headers=headers)
    with open(f'.//data//arxiv test//{link[21:]}.pdf', 'wb') as f:
        f.write(response.content)